In [1]:
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from astropy import units as u
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool
import pandas as pd
import numpy as np

from bokeh.transform import linear_cmap
from bokeh.palettes import Viridis256

In [2]:
# Target star coordinates (RA and Dec for HD 234683)
target_ra = 278.97016  # Right Ascension in degrees
target_dec = 54.840194  # Declination in degrees
target_coord = SkyCoord(ra=target_ra * u.deg, dec=target_dec * u.deg, frame="icrs")

In [3]:
print(Simbad.list_votable_fields())

--NOTES--

1. The parameter filtername must correspond to an existing filter. Filters include: B,V,R,I,J,K.  They are checked by SIMBAD but not astroquery.simbad

2. Fields beginning with rvz display the data as it is in the database. Fields beginning with rv force the display as a radial velocity. Fields beginning with z force the display as a redshift

3. For each measurement catalog, the VOTable contains all fields of the first measurement. When applicable, the first measurement is the mean one. 

Available VOTABLE fields:

bibcodelist(y1-y2)
biblio
cel
cl.g
coo(opt)
coo_bibcode
coo_err_angle
coo_err_maja
coo_err_mina
coo_qual
coo_wavelength
coordinates
dec(opt)
dec_prec
diameter
dim
dim_angle
dim_bibcode
dim_incl
dim_majaxis
dim_minaxis
dim_qual
dim_wavelength
dimensions
distance
distance_result
einstein
fe_h
flux(filtername)
flux_bibcode(filtername)
flux_error(filtername)
flux_name(filtername)
flux_qual(filtername)
flux_system(filtername)
flux_unit(filtername)
fluxdata(filtername)

In [4]:
Simbad.get_field_description ('flux(filtername)') 

value of the flux for the given filter


In [5]:
# Query SIMBAD for neighboring stars within 1 degree
simbad = Simbad()
simbad.TIMEOUT = 120  # Increase timeout for larger queries
simbad.add_votable_fields("ra", "dec", "flux(V)", "distance_result","plx", "plx_error")
result = simbad.query_region(target_coord, radius=1.5 * u.deg)

In [6]:
# Extract data into a Pandas DataFrame
stars = pd.DataFrame({
    "ra": result["RA"],
    "dec": result["DEC"],
    "magnitude": result["FLUX_V"],
    "distance": 3.26156*1000/result["PLX_VALUE"],  # Distance in lightyears
    "name": result["MAIN_ID"]
})

In [7]:
# stars = stars[stars["magnitude"]>1]

stars["size"] = 6*stars["magnitude"]/stars["magnitude"].max()
stars["shape"] = "circle"
stars["fill_color"] = "white"
stars["line_color"] = "white"
stars["fill_alpha"] = 1-stars["distance"]/10000
stars.loc[stars["fill_alpha"]<0,"fill_alpha"] = 0.0

# Convert RA and Dec to degrees
coords = SkyCoord(stars["ra"], stars["dec"], unit=(u.hourangle, u.deg))
stars["ra_deg"] = coords.ra.deg
stars["dec_deg"] = coords.dec.deg

In [8]:
stars = stars.dropna()

In [9]:
stars.loc[stars["name"]=="HD 234683","shape"]="star"
stars.loc[stars["name"]=="HD 234683","size"]=10

In [10]:
# Prepare data for Bokeh
source = ColumnDataSource(stars)

# Create the plot
p = figure(
    title="HD 234683 and Neighboring Stars",
    x_axis_label="Right Ascension (degrees)",
    y_axis_label="Declination (degrees)",
    tools="pan,wheel_zoom,reset,save",
    tooltips="""
        <div>
            <strong>Star:</strong> @name<br>
            <strong>RA:</strong> @ra<br>
            <strong>Dec:</strong> @dec<br>
            <strong>Magnitude:</strong> @magnitude<br>
            <strong>Distance:</strong> @distance{0,0} lightyears
        </div>
    """
)

# Add stars to the plot
p.scatter(
    x="ra_deg",
    y="dec_deg",
    size="size",
    marker="shape",
    fill_alpha="fill_alpha",
    source=source,
    fill_color = "fill_color",
    line_color = "line_color",
    
)

p.background_fill_color = "black"

# Remove gridlines
p.xgrid.grid_line_color = None  # Remove gridlines for x-axis
p.ygrid.grid_line_color = None  # Remove gridlines for y-axis


# Set view limits
zoom_level = 2
p.x_range.start = stars[stars["name"]=="HD 234683"]["ra_deg"].values[0]-1.0*zoom_level  # Minimum x-axis value
p.x_range.end = stars[stars["name"]=="HD 234683"]["ra_deg"].values[0]+1.0*zoom_level    # Maximum x-axis value
p.y_range.start = stars[stars["name"]=="HD 234683"]["dec_deg"].values[0]-0.5*zoom_level  # Minimum y-axis value
p.y_range.end = stars[stars["name"]=="HD 234683"]["dec_deg"].values[0]+0.5*zoom_level  # Maximum y-axis value

# Finalize the plot
show(p)
